In [1]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from gym import spaces
from pypylon import pylon
import time
from matplotlib import cm
import random

import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
from torchvision import models, transforms
import time
import warnings

import imquality.brisque as brisque

%matplotlib inline

env_closer = closer.Closer()

warnings.filterwarnings("ignore", category=FutureWarning)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class BaslerEnv(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    
    def __init__(self, 
                 file_path = r'D:\expos_time'):
        
        
        # Create an instant camera object with the camera device found first.
        self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.camera.Open()
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.ExposureTimeRaw.Value = 20000
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.camera.Width.Value = 417
        self.camera.Height.Value = 404
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        print(number_of_actions)
        
        self.resnet152 = models.resnet152(pretrained=True)
        modules=list(self.resnet152.children())[:-1]
        self.resnet152=nn.Sequential(*modules)
        for p in self.resnet152.parameters():
            p.requires_grad = False
        
        self.action_space = spaces.Discrete(number_of_actions)
        #self.observation_space = spaces.Box(0, np.inf, (21, ))
        self.observation_space = spaces.Tuple((spaces.Box(0, 2, (2048, )),
                                               spaces.Box(0, np.inf, (10, )), 
                                               spaces.Box(0, np.inf, (11, ))
                                              ))

        
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(3) 
        
        
    def take_features(self, img):
        preprocess = transforms.Compose([
                            transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        
        image = Image.fromarray(np.uint8(img)).convert('RGB')
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        
        # Run through the convolutional layers and resize the output.
        features_output = self.resnet152(input_batch)
        classifier_input = features_output.view(1, -1)
        
        return classifier_input.cpu().detach().numpy()[0]
        
    def basler(self, action):  
        ExposureTimeRaw = int(self.range_ex_time[action])
        print('ExposureTimeRaw:', ExposureTimeRaw) 
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(3) 
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
    
    def hist(self, image):
        hist, bin_edges =np.histogram(np.array(image).ravel())
        #image_inf = np.concatenate((hist, bin_edges))
        
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin >= 50 and max_value_bin <= 150:
            image_asses = 1
        elif (max_value_bin > 25 and max_value_bin < 50) or (max_value_bin > 150 and max_value_bin < 175):
            image_asses = - 0.01*round(brisque.score(image), 2)
        else:
            image_asses = -1
        return hist, bin_edges, image_asses
        
    def reset(self):
        self.lens_movement(48.16)
        self.camera.ExposureTimeRaw.Value = 20000
        time.sleep(3) 
        
        image = self.grab_image()
        hist, bins, _ = self.hist(image)
        
        obs_feat = self.take_features(image)
        
        obs = (obs_feat, hist, bins)
        
        return obs
    
    def step(self, action):
        self.basler(action)
        image = self.grab_image()
        hist, bins, image_asses = self.hist(image)
        print('Brisque:', image_asses)
        
        done = True
        if image_asses == -1:
            done = False
                 
        filepath = r'D:\expos_time'
        im = Image.fromarray(image)
        filename = filepath + "\img_%s_%s.png" % (str(action), str(image_asses))
        im.save(filename)
        
        obs_feat = self.take_features(image)
        
        obs = (obs_feat, hist, bins)

        return obs, image_asses, done, {}

In [3]:
import ray
from ray.tune.registry import register_env

def env_creator(env_config):
    return  BaslerEnv(env_config)

register_env("BaslerEnv", env_creator)

In [4]:
ray.init()

config={
        "env": BaslerEnv,
        "num_workers": 1,
        "entropy_coeff": 0.01,
        "rollout_fragment_length": 128,
        "train_batch_size": 128,
        "sgd_minibatch_size": 128,
        "lr": 0.00025,#tune.grid_search([0.01, 0.001, 0.0001]),
        # Smooth metrics over this many episodes.
        "metrics_smoothing_episodes": 10000,
    }

trainer = ppo.PPOTrainer(env= "BaslerEnv", config=config)

2021-05-13 14:47:35,506	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-13 14:47:37,476	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-05-13 14:47:37,478	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=49332) WARNING:tensorflow:From c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\tensorflow\python\compat\v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=49332) Instructions for updating:
(pid=49332) non-resource variables are not supported in the long term


(pid=49332) 417 404
(pid=49332) eCOMCaspErr: 0 0
(pid=49332) 148


(pid=49332) 2021-05-13 14:47:43,139	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 14:47:44,250	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 14:47:46,979	WARNING util.py:47 -- Install gputil for GPU system monitoring.


In [5]:
trainer.restore(r'C:\Users\CIG\ray_results\PPO_BaslerEnv_2021-05-13_11-20-16sf4cl1pg\checkpoint_41\checkpoint-41')

2021-05-13 14:47:47,077	INFO trainable.py:372 -- Restored on 10.16.102.19 from checkpoint: C:\Users\CIG\ray_results\PPO_BaslerEnv_2021-05-13_11-20-16sf4cl1pg\checkpoint_41\checkpoint-41
2021-05-13 14:47:47,077	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 41, '_timesteps_total': None, '_time_total': 41682.39471387863, '_episodes_total': 2859}


In [ ]:
for i in range(100):
    # Perform one iteration of training the policy with PPO\n",
    result = trainer.train()
    print(pretty_print(result))
    print('-------', i, '-------')
# #     print(evaluation_results)
    if i % 80 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

(pid=49332) 2021-05-13 14:47:53,859	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


(pid=49332) ExposureTimeRaw: 25000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 32000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 34000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 30000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 43000
(pid=49332) Brisque: 

(pid=49332) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=49332)   self.image = skimage.color.rgb2gray(self.image)


(pid=49332) Brisque: -0.31930000000000003
(pid=49332) ExposureTimeRaw: 100000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 50000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 28000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 180000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 285000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 38000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332)

(pid=49332) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=49332)   self.image = skimage.color.rgb2gray(self.image)


...

(pid=49332) Brisque: -0.3397
(pid=49332) ExposureTimeRaw: 60000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 28000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 11000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 125000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 55000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 85000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 285000


(pid=49332) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=49332)   self.image = skimage.color.rgb2gray(self.image)


(pid=49332) Brisque: -0.31120000000000003
(pid=49332) ExposureTimeRaw: 395000


(pid=49332) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=49332)   self.image = skimage.color.rgb2gray(self.image)


(pid=49332) Brisque: -0.29460000000000003
(pid=49332) ExposureTimeRaw: 22000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 34000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 445000
(pid=49332) Brisque: 1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 39000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 600000
(pid=49332) Brisque: 1
(pid=49332) ExposureTimeRaw: 335000
(pid=49332) Brisque: 1
(pid=49332) ExposureTimeRaw: 22000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 40000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 55000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 80000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 1000000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 36000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 90000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 10000
(pid=49332) Brisque: -1
(pid=49332) ExposureTimeRaw: 230000
(pid=49332) Brisque: 1
(pid=49332) 

In [ ]:
checkpoint = trainer.save()

In [ ]:
#tensorboard --logdir=~/ray_results